# How to use the conversation generator tools
In this notebook, we will show how to use the conversations generation tools to:
1. Generate and save one conversation with one emulated user
1. Generate and save one conversation with one emulated user given a scenario
1. Generate and save multiple conversations with different emulated users (with or without scenario)

## Introduction
The conversation generation tool is used with the orchestrator to create conversation data when none exists. It is the first step for both the inner loop and outer loop evaluation framework. There is an automated tool which helps generate multiple conversations, and there is a manual tool which allows users to converse with the orchestrator themselves. Simply put, it works in 3 steps:
1.	Generate an emulated user prompt. Optionally instruct it with a specific scenario. (This step can be skipped if using the manual tool.)
1.	Carry N turns of conversations between the emulated user and the orchestrator.
1.	Save the conversation to be used by the inner loop or outer loop evaluation framework.

## Emulated Users
To carry out a conversation with an orchestrator, we need to emulate a user. A user profile is generated using certain attributes like user's location, personality, preferences (using prompt template in customer_profile_template.py). Then the profile is inputted in the baseline prompt for the emulated user bot (using prompt template in emulated_customer_template.py). There are 2 baseline prompts, one with a scenario and one without. You can use the baseline prompt without the scenario and carry out a conversations with just the user profile's details. Or you can give it a specific scenario to carry out in the conversation.

The tools give 2 ways to emulate a user: The standard user generator and the random user generator.

### Standard User Generator
The standard user generator loops through a predefined list of user profiles (under eval\library\conversation_generator\user_generation\data\user_profiles.json). You can add to that file if you have specific profiles you want to emulate. 

### Random User Generator
The random user generator tool first builds a random user profile by looping through user profile data (under eval\library\conversation_generator\customer_profile_data). It randomly grabs an entry from each file and builds a user profile with them. You can add more to those files as well if needed



In [1]:
import sys
sys.path.append("../../")

In [2]:
from eval.library.conversation_generator.conversation import ConversationGenerator
from eval.library.conversation_generator.user_generation.standard_user import StandardUserGenerator
from eval.library.conversation_generator.user_generation.random_user import RandomUserGenerator

In [3]:
## Initialize the conversation Generator. 
# You can give it a maximum number of turns. The default number right now is set to 8
convo_gen = ConversationGenerator(max_turns=5) 

## Initialize the emulated user generator. We will showcase both standard user and randoem user 
standard_user_generator = StandardUserGenerator()
random_user_generator = RandomUserGenerator()

## Let's generate a user profile
standard_user_profile = standard_user_generator.generate_customer_profile()
random_user_profile = random_user_generator.generate_customer_profile()

c:\Users\ZITOURE\OneDrive - Microsoft\Documents\playGround\MercuryCrew\LLM-Inspect\weather-chatbot\eval\library\conversation_generator\user_generation\../customer_profile_data


In [4]:
## Let's take a look at these profiles
standard_user_profile
random_user_profile

{'prompt': '\nYou are a customer who is talking to a weather specialist named Handl. You must role play according         to the customer profile delineated by triple backticks.\n\nCustomer profile:\n```\nYou live in Marfa, Texas.\nYou always respond with just a single word..\nYou want to know if there are any severe weather alerts in your area.\n```\nLet Handl ask questions and learn about you.  Only share details about yourself when asked. ',
 'attributes': {'location': {'city': 'Marfa', 'state': 'Texas'},
  'weather_category': 'SEVERE_ALERTS'},
 'name': 'randomly generated user'}

#### Ganerate and Save one conversation with one emulated user
Let's generate and save a conversation with a standard user

In [5]:
context = convo_gen.generate_conversation(customer_profile=standard_user_profile)

------------------------------
Conversation_id: 90003d6d79ba4a99bbd1b1db2fa156df
------------------------------


[ Conversation Started ]

ASSISTANT: The user wants to know the weather forecast for Dallas this weekend. However, the provided weather data is for the date "2024-06-13" and does not cover the weekend. Therefore, I am unable to provide the forecast for Dallas this weekend based on the given data.


USER: What's the humidity level like?

Conversation Cont...

ASSISTANT: The current humidity level is 46%. Do you have any other questions about the weather?


USER: What's the wind speed?

Conversation Cont...

ASSISTANT: The current wind speed is 5.4 km/h. Do you have any other questions about the weather?


USER: Any chances of rain?

Conversation Cont...

ASSISTANT: assistant: Based on the provided weather data for the date "2024-06-13", there is no precipitation expected, which means there are no chances of rain. Do you have any other questions about the weather?


USER: UV index? High or low?

Conversation Cont...
...This is turn 4, and next turn will reach the max_turns,
we set for th

In [6]:
## Choose a log location. I will create a log folder under this notebooks folder
location = "logs"
convo_gen.save_conversation(context=context, log_location=location)

Complete log saved to logs\log_2024_06_13_15_08_21.txt.
Condensed log saved to logs\log_2024_06_13_15_08_21_condensed.xlsx.


#### Ganerate and Save one conversation with one emulated user and a scenario prompt
Let's generate and save a conversation with a random user profile

In [7]:
scenario = "You are a troll. You must not mention anything or any word related to weather no matter what. If the assistant talks about the weather, completely ignore it and talking about or ask question about anything else."
context = convo_gen.generate_conversation(customer_profile=standard_user_profile, scenario_prompt=scenario)

------------------------------
Conversation_id: 6470b596609b43cd94a1d5a2763906f9
------------------------------
[ Conversation Started ]

ASSISTANT: As an AI, I don't have personal experiences or preferences, so I don't have a favorite color. However, I'm here to help you. If you need information about the weather, could you please provide your geographical location? I need to know your city and country, and if possible, your zip code or state/province.


USER: What's the fastest land animal?

Conversation Cont...

ASSISTANT: The fastest land animal is the cheetah, which can reach speeds up to 60-70 miles per hour. However, if you need information about the weather, could you please provide your geographical location? I need to know your city and country, and if possible, your zip code or state/province.


USER: Ever tasted a cheeseburger? They're delish!

Conversation Cont...

ASSISTANT: As an AI, I don't have the ability to taste or eat food. But I'm glad to hear you enjoy cheeseburg

In [8]:
## you can pretty print conversations as well if needed
convo_gen.print_conversation(context)

Conversation Id: 6470b596609b43cd94a1d5a2763906f9
ASSISTANT: Hello! How can I help you?
USER: Hey there, Handl! What's your favorite color?
ASSISTANT: As an AI, I don't have personal experiences or preferences, so I don't have a favorite color. However, I'm here to help you. If you need information about the weather, could you please provide your geographical location? I need to know your city and country, and if possible, your zip code or state/province.
USER: What's the fastest land animal?
ASSISTANT: The fastest land animal is the cheetah, which can reach speeds up to 60-70 miles per hour. However, if you need information about the weather, could you please provide your geographical location? I need to know your city and country, and if possible, your zip code or state/province.
USER: Ever tasted a cheeseburger? They're delish!
ASSISTANT: As an AI, I don't have the ability to taste or eat food. But I'm glad to hear you enjoy cheeseburgers! If you need information about the weather, 

We generated and saved conversations under logs folder in text format and excel format.

#### Generate and save multiple conversations with different emulated users

We will generate and save multiple conversations in different files. Let's define a dataframe with 2 columns, a scenario prompt column and a num_convo column to denote how many conversations to generate for that scenario

In [21]:
import pandas as pd
data_convo = {'scenario_prompt': ['', scenario],
        'num_convo': [1, 2]}
df = pd.DataFrame(data_convo)
print(df)

                                     scenario_prompt  num_convo
0                                                             1
1  You are a troll. You must not mention anything...          2


We have 2 entries here. We will generate one conversation without a scenario and 2 conversations with the scenario we defined above
You can define your own dataframe and load from a csv file if needed.

In [ ]:
# Updating max turn to speed this up 
convo_gen.max_turns = 2
for scenario, num_conversations in df.values:
    for i in range(num_conversations):
        print(f'Generating conversation {i + 1} of {num_conversations}\n')
        # Generate customer profile
        customer_profile = standard_user_generator.generate_customer_profile()

        # Generate conversation
        context = convo_gen.generate_conversation(customer_profile, scenario)
        # Save conversation
        convo_gen.save_conversation(context=context,log_location=location)